In [2]:

import os
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [24]:
train_df = pd.read_csv("../project/input/train.csv")
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [7]:
emb_index = {}
emb_file = open('../project/input/embeddings/glove.840B.300d/glove.840B.300d.txt')
for line in tqdm(emb_file):
    emb_values = line.split(" ")
    word = emb_values[0]
    vecs = np.asarray(emb_values[1:], dtype='float32')
    emb_index[word] = vecs
emb_file.close()

print('Found %s word vectors.' % len(emb_index))

2196017it [02:00, 18241.99it/s]

Found 2196016 word vectors.


In [8]:
TIME_STEP = 30
EMB_VEC_SIZE = 300
BATCH_SIZE = 128
def text_to_vec(text):
    init_emb = np.zeros(EMB_VEC_SIZE)
    text = text[:-1].split()[:TIME_STEP]
    embeds = [emb_index.get(x, init_emb) for x in text]
    embeds+= [init_emb] * (TIME_STEP - len(embeds))
    return np.array(embeds)


In [9]:

def train_gen(train_df, shuffle=False):
    n_batches = math.ceil(len(train_df) / BATCH_SIZE)
    while True: 
        if(shuffle==True):
            train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*BATCH_SIZE:(i+1)*BATCH_SIZE, 1]
            text_arr = np.array([text_to_vec(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*BATCH_SIZE:(i+1)*BATCH_SIZE])

In [10]:
from keras.models import  Model
from keras.layers import LSTM, Dense, Bidirectional, Input, CuDNNLSTM
from tensorflow.python.client import device_lib

Using TensorFlow backend.


In [11]:
def get_available_gpus():
        local_device_protos = device_lib.list_local_devices()
        return [x.name for x in local_device_protos if x.device_type == 'GPU']
    
def get_base_model(input):

  num_gpus = get_available_gpus()
  #print(len(num_gpus))
  #Build LSTM network

  if(len(num_gpus)>0):
    first_lstm = Bidirectional(CuDNNLSTM(64, return_sequences=True, name='lstm1'))(input)
    second_lstm = Bidirectional(CuDNNLSTM(64, name='lstm2'))(first_lstm)
  else:  
    first_lstm = Bidirectional(LSTM(64, return_sequences=True, name='lstm1'))(input)
    second_lstm = Bidirectional(LSTM(64, name='lstm2'))(first_lstm)
  outputs = Dense(1, activation="sigmoid", name='last')(second_lstm)
  
  #create model LSTM+dense
  l_model = Model(input, outputs) 
  
  return(l_model)

In [12]:
features = Input(shape=(TIME_STEP, EMB_VEC_SIZE))
model = get_base_model(features)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [17]:

train_generator = train_gen(train_df, True)
validation_generator = train_gen(val_df, False)
model.fit_generator(train_generator,
                    steps_per_epoch=int(len(train_df) / BATCH_SIZE),
                    validation_data=validation_generator,
                    validation_steps = int(len(val_df) / BATCH_SIZE),
                    epochs=1,
                    verbose=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
9183/9183 [==============================] - 2888s 314ms/step - loss: 0.1136 - acc: 0.9554 - val_loss: 0.1047 - val_acc: 0.9582


In [39]:
def test_gen(train_df):
    n_batches = math.ceil(len(train_df) / BATCH_SIZE)
    for i in range(n_batches):
            texts = train_df.iloc[i*BATCH_SIZE:(i+1)*BATCH_SIZE, 1]
            text_arr = np.array([text_to_vec(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*BATCH_SIZE:(i+1)*BATCH_SIZE])

In [77]:
y_test = []
y_preds = []
for x, y in tqdm(test_gen(val_df)):
    y_preds.extend(model.predict(x))
    y_test.extend(y)


0it [00:00, ?it/s]
1it [00:00,  5.07it/s]
2it [00:00,  4.96it/s]
3it [00:00,  4.90it/s]
4it [00:00,  5.08it/s]
5it [00:01,  4.98it/s]
6it [00:01,  4.94it/s]
7it [00:01,  5.07it/s]
8it [00:01,  4.98it/s]
9it [00:01,  4.93it/s]
10it [00:02,  5.04it/s]
11it [00:02,  4.95it/s]
12it [00:02,  4.88it/s]
13it [00:02,  5.03it/s]
14it [00:02,  4.95it/s]
15it [00:03,  4.90it/s]
16it [00:03,  5.07it/s]
17it [00:03,  4.98it/s]
18it [00:03,  4.91it/s]
19it [00:03,  5.04it/s]
20it [00:04,  4.94it/s]
21it [00:04,  4.88it/s]
22it [00:04,  5.03it/s]
23it [00:04,  5.00it/s]
24it [00:04,  4.91it/s]
25it [00:05,  5.02it/s]
26it [00:05,  4.98it/s]
27it [00:05,  4.90it/s]
28it [00:05,  4.71it/s]
29it [00:05,  4.91it/s]
30it [00:06,  4.86it/s]
31it [00:06,  4.80it/s]
32it [00:06,  4.92it/s]
33it [00:06,  4.86it/s]
34it [00:06,  4.79it/s]
35it [00:07,  4.94it/s]
36it [00:07,  4.94it/s]
37it [00:07,  4.88it/s]
38it [00:07,  4.81it/s]
39it [00:07,  4.94it/s]
40it [00:08,  4.89it/s]
41it [00:08,  4.84it/s]
42it 

296it [01:04,  4.23it/s]
297it [01:05,  4.35it/s]
298it [01:05,  4.42it/s]
299it [01:05,  4.35it/s]
300it [01:05,  4.30it/s]
301it [01:05,  4.27it/s]
302it [01:06,  4.39it/s]
303it [01:06,  4.32it/s]
304it [01:06,  4.29it/s]
305it [01:06,  4.27it/s]
306it [01:07,  4.41it/s]
307it [01:07,  4.43it/s]
308it [01:07,  4.37it/s]
309it [01:07,  4.33it/s]
310it [01:08,  4.30it/s]
311it [01:08,  4.42it/s]
312it [01:08,  4.33it/s]
313it [01:08,  4.29it/s]
314it [01:08,  4.26it/s]
315it [01:09,  4.37it/s]
316it [01:09,  4.39it/s]
317it [01:09,  4.34it/s]
318it [01:09,  4.28it/s]
319it [01:10,  4.25it/s]
320it [01:10,  4.37it/s]
321it [01:10,  4.32it/s]
322it [01:10,  4.28it/s]
323it [01:11,  4.25it/s]
324it [01:11,  4.22it/s]
325it [01:11,  4.37it/s]
326it [01:11,  4.30it/s]
327it [01:12,  4.25it/s]
328it [01:12,  4.25it/s]
329it [01:12,  4.38it/s]
330it [01:12,  4.44it/s]
331it [01:12,  4.34it/s]
332it [01:13,  4.27it/s]
333it [01:13,  4.23it/s]
334it [01:13,  4.36it/s]
335it [01:13,  4.28it/s]


588it [02:12,  4.55it/s]
589it [02:12,  4.58it/s]
590it [02:12,  4.48it/s]
591it [02:12,  4.36it/s]
592it [02:13,  4.53it/s]
593it [02:13,  4.61it/s]
594it [02:13,  4.53it/s]
595it [02:13,  4.48it/s]
596it [02:13,  4.60it/s]
597it [02:14,  4.53it/s]
598it [02:14,  4.46it/s]
599it [02:14,  4.41it/s]
600it [02:14,  4.52it/s]
601it [02:15,  4.55it/s]
602it [02:15,  4.47it/s]
603it [02:15,  4.43it/s]
604it [02:15,  4.53it/s]
605it [02:15,  4.59it/s]
606it [02:16,  4.49it/s]
607it [02:16,  4.41it/s]
608it [02:16,  4.57it/s]
609it [02:16,  4.58it/s]
610it [02:17,  4.48it/s]
611it [02:17,  4.43it/s]
612it [02:17,  4.55it/s]
613it [02:17,  4.58it/s]
614it [02:17,  4.51it/s]
615it [02:18,  4.48it/s]
616it [02:18,  4.61it/s]
617it [02:18,  4.54it/s]
618it [02:18,  4.45it/s]
619it [02:19,  4.42it/s]
620it [02:19,  4.55it/s]
621it [02:19,  4.55it/s]
622it [02:19,  4.45it/s]
623it [02:19,  4.40it/s]
624it [02:20,  4.54it/s]
625it [02:20,  4.59it/s]
626it [02:20,  4.48it/s]
627it [02:20,  4.41it/s]


880it [03:18,  4.42it/s]
881it [03:18,  4.37it/s]
882it [03:19,  4.34it/s]
883it [03:19,  4.46it/s]
884it [03:19,  4.40it/s]
885it [03:19,  4.36it/s]
886it [03:20,  4.32it/s]
887it [03:20,  4.46it/s]
888it [03:20,  4.49it/s]
889it [03:20,  4.41it/s]
890it [03:20,  4.37it/s]
891it [03:21,  4.32it/s]
892it [03:21,  4.48it/s]
893it [03:21,  4.42it/s]
894it [03:21,  4.35it/s]
895it [03:22,  4.33it/s]
896it [03:22,  4.43it/s]
897it [03:22,  4.38it/s]
898it [03:22,  4.32it/s]
899it [03:22,  4.32it/s]
900it [03:23,  4.28it/s]
901it [03:23,  4.42it/s]
902it [03:23,  4.38it/s]
903it [03:23,  4.33it/s]
904it [03:24,  4.29it/s]
905it [03:24,  4.43it/s]
906it [03:24,  4.39it/s]
907it [03:24,  4.35it/s]
908it [03:25,  4.34it/s]
909it [03:25,  4.47it/s]
910it [03:25,  4.48it/s]
911it [03:25,  4.41it/s]
912it [03:25,  4.37it/s]
913it [03:26,  4.33it/s]
914it [03:26,  4.45it/s]
915it [03:26,  4.40it/s]
916it [03:26,  4.36it/s]
917it [03:27,  4.31it/s]
918it [03:27,  4.44it/s]
919it [03:27,  4.38it/s]


In [100]:
y1 = np.array(y_preds)
y2 = np.array(y_test)
y11 = y1.squeeze()
y1 = y11 > 0.5
y1 = y1.astype(np.int)        
score = f1_score(y2, y1)
print(score)    

0.6076005302695537
